In [1]:
!git clone https://github.com/acmilannesta/Adult_readmission

Cloning into 'Adult_readmission'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 52 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [0]:
!pip install tsfresh
!pip install pandas==0.22

In [0]:
import pandas as pd
from tsfresh import extract_relevant_features, extract_features

In [0]:
lab = pd.read_csv('Adult_readmission/PA_0501.csv', parse_dates=['SPECIMEN_COLLECT_DT', 'TRANSPLANT_DT', 'TRANSPLANT_DISCHARGE_DT'])
cond1 = (lab['TRANSPLANT_DISCHARGE_DT'] >= lab['SPECIMEN_COLLECT_DT'])
cond2 = ((lab['TRANSPLANT_DT'] - lab['SPECIMEN_COLLECT_DT']).map(lambda x: x.days) <= 90)
lab = lab[cond1 & cond2]
y = pd.read_sas('Adult_readmission/combined_vars_04092019.sas7bdat')[['PERSON_ID', 'CODE_REHOSP']]

**Biomarker list created**

"markerlist": biomarkers with multiple time points

'singlemarker': biomarkers with the last time point taken before transplantation

In [0]:
markerlist = ['CREATININE',
           'GLUCOSE',
           'MAGNESIUM',
           'CHLORIDE',
           'ANION_GAP__BLOOD',
           'ALKALINE_PHOSPHATASE',
           'ALBUMIN',
           'BUN__BLOOD_UREA_NITROGEN_',
           'CO2',
           'HEMATOCRIT',
           'HEMOGLOBIN',
           'POTASSIUM_PLASMA',
           'SODIUM_PLASMA',
           'UREA_CREATININE_RATIO',
             ]
singlemarker = ['DRUG_SCREEN', 
                'GLUCOSE_URINE_QUALITATIVE',
                'NITRITE_URINE_QUALITATIVE', 
                'PROTEIN_URINE_QUALITATIVE',
                'HBA1C__HEMOGLOBIN_A1C_',
                'INDIRECT_BILIRUBIN',
                'NON_HDL_CHOLESTEROL',
                'PROGRAF__TACROLIMUS_',
                'PT_RESULT',
                'URIC_ACID',
               ]

In [0]:
biomarker_extract = y['PERSON_ID'].to_frame()
for marker in markerlist:
  biomarker = lab[['PERSON_ID', 'SPECIMEN_COLLECT_DT', marker]].dropna().sort_values(['PERSON_ID', 'SPECIMEN_COLLECT_DT'])
  biomarker['time'] = biomarker.groupby('PERSON_ID').cumcount()
  y_biomarker = biomarker['PERSON_ID'].drop_duplicates().to_frame().merge(y, on='PERSON_ID', how='left')
  y_biomarker.index=y_biomarker['PERSON_ID']  
  f = extract_relevant_features(biomarker.drop('SPECIMEN_COLLECT_DT', 1), y_biomarker['CODE_REHOSP'], column_id='PERSON_ID', column_sort='time')
  f['PERSON_ID'] = f.index
  biomarker_extract = biomarker_extract.merge(f, on='PERSON_ID', how='left')

Feature Extraction: 100%|██████████| 5/5 [01:33<00:00, 18.91s/it]
 'CREATININE__fft_coefficient__coeff_61__attr_"angle"'
 'CREATININE__fft_coefficient__coeff_61__attr_"imag"'
 'CREATININE__fft_coefficient__coeff_61__attr_"real"'
 'CREATININE__fft_coefficient__coeff_62__attr_"abs"'
 'CREATININE__fft_coefficient__coeff_62__attr_"angle"'
 'CREATININE__fft_coefficient__coeff_62__attr_"imag"'
 'CREATININE__fft_coefficient__coeff_62__attr_"real"'
 'CREATININE__fft_coefficient__coeff_63__attr_"abs"'
 'CREATININE__fft_coefficient__coeff_63__attr_"angle"'
 'CREATININE__fft_coefficient__coeff_63__attr_"imag"'
 'CREATININE__fft_coefficient__coeff_63__attr_"real"'
 'CREATININE__fft_coefficient__coeff_64__attr_"abs"'
 'CREATININE__fft_coefficient__coeff_64__attr_"angle"'
 'CREATININE__fft_coefficient__coeff_64__attr_"imag"'
 'CREATININE__fft_coefficient__coeff_64__attr_"real"'
 'CREATININE__fft_coefficient__coeff_65__attr_"abs"'
 'CREATININE__fft_coefficient__coeff_65__attr_"angle"'
 'CREATININE__f

In [0]:
for marker in singlemarker:
    tmp = lab[['PERSON_ID', marker]].dropna()
    assert(tmp.drop_duplicates().shape[0] == tmp.shape[0])
    biomarker_extract = biomarker_extract.merge(tmp, on='PERSON_ID', how='left')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
biomarker_extract.to_csv('drive/My Drive/Adult readmission/feature_extracted_90days.csv', index=False)